In [1]:
import os

source_file='data_thchs30'
def source_get(source_file):
    train_file='./'+source_file+'/data'
    label_lst=[]
    wav_lst=[]
    for root, dirs, files in os.walk(train_file):
        for file in files:
            if file.endswith('.wav') or file.endswith('.WAV'):
                wav_file=os.sep.join([root,file])
                label_file=wav_file+'.trn'
                wav_lst.append(wav_file)
                label_lst.append(label_file)
    return label_lst, wav_lst
# label_lst 是类标文件，即.trn为后缀的文件
# wav_lst 是语音文件，即.wav是后缀的文件
# 这两个文件是训练数据
label_lst, wav_lst=source_get(source_file)
print('length')

length


In [2]:
#进行验证数据读取过程中有没有问题
for i in range(len(label_lst)):
    wavname=(wav_lst[i].split('/')[-1]).split('.')[0]
    labelname = (label_lst[i].split('/')[-1]).split('.')[0]
    if wavname != labelname:
        print('error')

In [3]:
def read_label(label_file):
    with open(label_file, 'r', encoding='utf8') as f:
        data=f.readlines()
        return data[1]
#label_file当中第一行存储的是中文字符，第二行存储的是对应的语音，第三行是对应的语素

In [4]:
# 总共有13388条语音，对应着13388条拼音记录
# gen_label_data函数返回这13388条语音记录对应拼音
def gen_label_data(label_lst):
    label_data=[]
    for label_file in label_lst:
        pny=read_label(label_file)
        label_data.append(pny.strip('\n'))
    return label_data

label_data=gen_label_data(label_lst)
label_data[1]

'ta1 jin3 ping2 yao1 bu4 de5 li4 liang4 zai4 yong3 dao4 shang4 xia4 fan1 teng2 yong3 dong4 she2 xing2 zhuang4 ru2 hai3 tun2 yi4 zhi2 yi3 yi4 tou2 de5 you1 shi4 ling3 xian1'

In [5]:
#通过统计语料获得有多少发音，并把这些发音存储到vocab中
# vocab就是一个字典的功能，汉语拼音和对应的索引构成字典
def mk_vocab(label_data):
    vocab=[]
    for line in label_data:
        line=line.split(' ')
        for pny in line:
            if pny not in vocab:
                vocab.append(pny)
    vocab.append('_')
    return vocab

vocab =mk_vocab(label_data)

In [6]:
# 基于vocab字典获取label（拼音）对应的id
def word2id(line, vocab):
    return [vocab.index(pny) for pny in line.split(' ')]#以空格为分隔符

In [7]:
total_nums = 100
batch_size = 20
batch_num = total_nums // batch_size
epochs = 50

print(len(vocab))
shuffle_list = [i for i in range(100)]

1209


In [8]:
#打乱数据的顺序，我们通过查询乱序的索引，确定训练数据的顺序
from random import shuffle
shuffle_list=[i for i in range(total_nums)]
shuffle(shuffle_list)

In [9]:
def wav_padding(wav_data_lst):
    wav_lens = [data.shape[0] for data in wav_data_lst]
    wav_max_len = max(wav_lens)
    wav_lens = np.array([leng//8 for leng in wav_lens])
    new_wav_data_lst = np.zeros((len(wav_data_lst), wav_max_len, 120,  1))
    for i in range(len(wav_data_lst)):
        new_wav_data_lst[i, :wav_data_lst[i].shape[0], :, 0] = wav_data_lst[i]
    return new_wav_data_lst, wav_lens

In [10]:
def label_padding(label_data_lst):
    label_lens = np.array([len(label) for label in label_data_lst])
    max_label_len = max(label_lens)
    new_label_data_lst = np.zeros((len(label_data_lst), max_label_len))
    for i in range(len(label_data_lst)):
        new_label_data_lst[i][:len(label_data_lst[i])] = label_data_lst[i]
    return new_label_data_lst,label_lens

In [11]:
import librosa
import numpy as np

def data_generator(batch_size, shuffle_list, wav_lst, label_data, vocab):
    for i in range(len(wav_lst)//batch_size):
        wav_data_lst=[]
        label_data_lst=[]
        begin=i*batch_size
        end=begin+batch_size
        sub_list=shuffle_list[begin:end]
        for index in sub_list:
            x,r=librosa.load(wav_lst[index])
            mfcc=librosa.feature.mfcc(x,r,n_mfcc=120).T
            pad_mfcc=np.zeros((mfcc.shape[0]//8*8+8,mfcc.shape[1]))
            pad_mfcc[:mfcc.shape[0],:]=mfcc
            label=word2id(label_data[index],vocab)
            wav_data_lst.append(pad_mfcc)
            label_data_lst.append(label)
        pad_wav_data,input_length=wav_padding(wav_data_lst)
        print(pad_wav_data.shape)
        pad_label_data,label_length=label_padding(label_data_lst)
        inputs = {'the_inputs': pad_wav_data,
                  'the_labels': pad_label_data,
                  'input_length': input_length,
                  'label_length': label_length,
                 }
        outputs = {'ctc': np.zeros(pad_wav_data.shape[0],)} 
#         return inputs, outputs
        yield inputs, outputs


In [12]:
x,r=librosa.load(wav_lst[0])
mfcc=librosa.feature.mfcc(x,r,n_mfcc=120).T
print(mfcc.shape)
# 进行了三次卷积核pooling因此数据大小改为原有的1/8。所以数据的长宽需要被8整除
pad_mfcc=np.zeros((mfcc.shape[0]//8*8+8,mfcc.shape[1]))
print(pad_mfcc.shape)
pad_mfcc[:mfcc.shape[0],:]=mfcc
print(pad_mfcc[0])

(336, 120)
(344, 120)
[-4.46572715e+02  9.26332211e+01  9.99465923e-01  2.03506718e+01
  2.65053779e+01  5.88762892e+00  3.02444206e+00  3.34910065e+00
  1.59525124e+01  2.29271629e+01  9.26783102e+00  7.56682440e+00
  1.22366358e+01  9.83432997e+00  1.33641370e+01  1.25091871e+01
  3.59305888e+00  1.81684241e+00  4.09812087e+00  4.41741694e+00
  2.08384104e+00 -1.58594964e+00  1.41728099e+00  6.32762927e+00
  4.87731823e+00  1.17850904e+00 -2.57193895e+00 -3.01324504e-01
  4.61401160e+00 -2.47471393e+00 -5.61845880e+00  3.66761102e+00
  9.51778869e-01 -5.19791760e+00  1.35770385e+00  2.50211155e+00
 -9.75789262e-01  1.46323353e+00 -4.33361416e-01 -3.72260400e+00
 -1.51385918e+00 -1.48119765e+00 -4.15058199e-01  6.31708049e+00
  7.39308611e+00 -3.36807754e-01 -4.05350348e+00  2.41326216e+00
  1.00836643e+01  7.47808430e+00 -2.07632364e+00 -3.89542263e+00
  5.02759406e-01  1.32004771e-01  3.14348593e+00  1.10525731e+01
  7.42853101e+00  8.25944422e-02  3.14807913e+00  3.93206427e+00
 -7

In [13]:
inputs, outputs=next(data_generator(batch_size, shuffle_list, wav_lst, label_data, vocab))
# 这个ctc.shape[0]实际上是batch大小，这个输出结果实际上是(batch_size,None)。这里None是因为每一条训练语音可采样不同长度个帧
# 因此对应不同长度的类标
outputs['ctc'].shape

(20, 512, 120, 1)


(20,)

In [14]:
import keras
from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D
from keras.layers import Reshape, Dense, Lambda
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model
from keras.utils import multi_gpu_model

ModuleNotFoundError: No module named 'keras'

In [ ]:
def conv2d(size):
    return Conv2D(size, (3,3), use_bias=True, activation='relu',
        padding='same', kernel_initializer='he_normal')

In [ ]:
def norm(x):
    return BatchNormalization(axis=-1)(x)

In [ ]:
def maxpool(x):
    return MaxPooling2D(pool_size=(2,2), strides=None, padding="valid")(x)

In [ ]:
def dense(units, activation="relu"):
    return Dense(units, activation=activation, use_bias=True,
        kernel_initializer='he_normal')

In [ ]:
# x.shape=(none, none, none)
# output.shape = (1/2, 1/2, 1/2)
def cnn_cell(size, x, pool=True):
    x = norm(conv2d(size)(x))
    x = norm(conv2d(size)(x))
    if pool:
        x = maxpool(x)
    return x

In [ ]:
# args: self.labels, self.outputs, self.input_length, self.label_length
def ctc_lambda(args):
    labels, y_pred, input_length, label_length = args
    y_pred = y_pred[:, :, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
class Amodel():
    """docstring for Amodel."""
    def __init__(self, vocab_size):
        super(Amodel, self).__init__()
        self.vocab_size = vocab_size
        self._model_init()
        self._ctc_init()
        self.opt_init()

    def _model_init(self):
        self.inputs = Input(name='the_inputs', shape=(None, 120, 1))
        self.h1 = cnn_cell(32, self.inputs)
        self.h2 = cnn_cell(64, self.h1)
        self.h3 = cnn_cell(128, self.h2)
        self.h4 = cnn_cell(128, self.h3, pool=False)
        # 120 / 8 * 128 = 1920
        self.h6 = Reshape((-1, 1920))(self.h4)
        self.h7 = dense(256)(self.h6)
        self.outputs = dense(self.vocab_size, activation='softmax')(self.h7)
#         self.model = Model(inputs=self.inputs, outputs=self.outputs)

    def _ctc_init(self):
        self.labels = Input(name='the_labels', shape=[None], dtype='float32')
        self.input_length = Input(name='input_length', shape=[1], dtype='int64')
        self.label_length = Input(name='label_length', shape=[1], dtype='int64')
#         当要使用一些keras.layers中没有的层的时候，需想办法将其转换成keras中的Layer。
#         其中Lambda函数就是一种方法。定义一个squeeze层，其中x是上一层的输出，
#         Lambda(keras.backend.squeeze)(x)
        self.loss_out = Lambda(ctc_lambda, output_shape=(1,), name='ctc')\
            ([self.labels, self.outputs, self.input_length, self.label_length])
        self.ctc_model = Model(inputs=[self.labels, self.inputs,
            self.input_length, self.label_length], outputs=self.loss_out)
        
    def opt_init(self):
        opt = Adam(lr = 0.0008, beta_1 = 0.9, beta_2 = 0.999, decay = 0.01, epsilon = 10e-8)
        #self.ctc_model=multi_gpu_model(self.ctc_model,gpus=2)
        self.ctc_model.compile(loss={'ctc': lambda y_true, output: output}, optimizer=opt)
# loss: 字符串（目标函数名）或目标函数或 Loss 实例。 如果模型具有多个输出，
# 则可以通过传递损失函数的字典或列表，在每个输出上使用不同的损失。
# 模型将最小化的损失值将是所有单个损失的总和

In [ ]:
am=Amodel(1176)
am.ctc_model.summary()

In [ ]:
total_nums = 100
batch_size = 20
batch_num = total_nums // batch_size
epochs = 50
source_file = 'data_thchs30'
label_lst, wav_lst = source_get(source_file)
label_data = gen_label_data(label_lst[:100])
vocab = mk_vocab(label_data)
vocab_size = len(vocab)

print(vocab_size)

shuffle_list = [i for i in range(100)]

In [ ]:
am = Amodel(vocab_size)

for k in range(epochs):
    print('this is the', k+1, 'th epochs trainning !!!')
    #shuffle(shuffle_list)
    batch = data_generator(batch_size, shuffle_list, wav_lst, label_data, vocab)
    am.ctc_model.fit_generator(batch, steps_per_epoch=batch_num, epochs=1)